# Sprint 3 — Webinar 7 Teórico  
## Introducción a SQL con DuckDB (desde el navegador) — Dataset *Bookstore*

<div style="text-align: center; margin-top: 10px;">
  <img src="https://raw.githubusercontent.com/duckdb/duckdb-web/main/static/images/logo/DuckDB_Logo.png" width="260">
</div>


> **Duración sugerida:** 100 minutos  
> **Herramienta (solo navegador):** DuckDB en https://sql-workbench.com/  
> **Dataset:** 5 archivos CSV (Bookstore) que tú subirás a GitHub (raw links)

---

## Agenda (100 min)
1. Introducción a SQL y a DuckDB (10 min)  
2. Entendiendo el esquema relacional del dataset (20 min)  
3. Llaves primarias, foráneas y relaciones (10 min)  
4. Inspeccionar “tablas” desde CSV en DuckDB (10 min)  
5. Consultas básicas: `SELECT`, `WHERE`, `ORDER BY`, `LIMIT` (20 min)  
6. Agregaciones y `GROUP BY` (+ `HAVING`) (25 min)  
7. Tipos, funciones útiles y limpieza (10–15 min)  
8. Buenas prácticas para SQL legible (10 min)  
**Bonus si hay tiempo:** funciones de ventana (5–10 min)

---

## Preparación previa (Instructor)
Este webinar se dicta **solo con DuckDB**. Antes de clase, sube los CSV a un repositorio en GitHub, por ejemplo:

- `data/duckdb_bookstore_customers.csv`
- `data/duckdb_bookstore_products.csv`
- `data/duckdb_bookstore_orders.csv`
- `data/duckdb_bookstore_order_items.csv`
- `data/duckdb_bookstore_payments.csv`

Luego toma el enlace **RAW** de cada archivo (GitHub → abrir CSV → botón “Raw” → copiar URL).


### Objetivos de aprendizaje
Al finalizar, el estudiante podrá:

- Explicar qué es SQL y para qué se usa en analítica.
- Ejecutar consultas básicas con DuckDB desde el navegador.
- Cargar datos desde CSV (por URL) con `read_csv_auto`.
- Filtrar, ordenar, agregar y unir datos con `JOIN`.
- Formular preguntas analíticas simples (KPIs básicos) a partir del dataset.
- Escribir SQL más limpio (alias, sangría, nombres consistentes).

### ✅ Reglas del juego (para estudiantes)
- Aquí **no hay Python**. Todo lo hacemos en el editor SQL del navegador.
- Cada consulta debe ser **copiada/pegada** y ejecutada para ver resultados.
- Si algo falla, leemos el error: en SQL, el error es parte del aprendizaje.


### Conexión a la base de datos (DuckDB en el navegador)
1. Abre https://sql-workbench.com/  
2. Selecciona **DuckDB** como motor (si te lo pide).  
3. En el editor, ejecuta una consulta de prueba:

```sql
SELECT 1 AS ok;
```

Si ves una tabla con `ok = 1`, estás listo/a.


## Introducción a SQL (Structured Query Language)

### ¿Qué es SQL?
SQL es el lenguaje estándar para **consultar** y **transformar** datos almacenados en estructuras tabulares (tablas).  
Como analistas, lo usamos para responder preguntas del negocio: “¿qué pasó?”, “¿cuánto?”, “¿dónde?”, “¿cuándo?”, “¿qué segmento?”.

### ¿Qué es DuckDB y por qué lo usamos?
DuckDB es un motor analítico “tipo OLAP” que corre localmente (o embebido) y es excelente para:
- Analítica rápida sobre archivos (CSV/Parquet)
- Consultas tipo “data warehouse” en pequeño/mediano
- Ejecutar SQL sin instalar un servidor

En este webinar lo usamos porque permite algo muy cómodo:
```sql
SELECT * FROM read_csv_auto('https://.../archivo.csv');
```

### ¿Cómo funciona una consulta? (modelo mental)
Piensa en SQL como una tubería:

1) **FROM**: ¿de dónde vienen los datos?  
2) **WHERE**: ¿qué filas me interesan?  
3) **GROUP BY**: ¿cómo agrupo? (si aplica)  
4) **SELECT**: ¿qué columnas/calculos quiero?  
5) **ORDER BY**: ¿en qué orden los veo?

> Importante: en el código se escribe `SELECT ... FROM ... WHERE ...`,  
> pero el motor “piensa” en ese orden lógico (FROM → WHERE → GROUP → SELECT → ORDER).

### Comandos fundamentales (hoy)
- `SELECT` (ver columnas / cálculos)
- `FROM` (fuente de datos: tabla o CSV)
- `WHERE` (filtro)
- `ORDER BY` (orden)
- `LIMIT` (muestra)
- `GROUP BY` + funciones agregadas (`COUNT`, `SUM`, `AVG`, etc.)
- `JOIN` (unir tablas)


### 💡 Tips iniciales (para sobrevivir el primer SQL)
- Empieza con: `SELECT * FROM ... LIMIT 10;`
- Si no sabes el nombre exacto de una columna: primero mira una muestra.
- Usa alias cortos para tablas: `customers c`, `orders o`.
- No intentes hacerlo perfecto a la primera: itera.
- Escribe SQL como si alguien más lo fuera a leer.


## 0) Cargar el dataset desde GitHub (5–8 min)

En DuckDB, podemos “leer” un CSV directamente desde una URL RAW.
Pero para no repetir `read_csv_auto` en cada consulta, crearemos **VIEWS** (vistas):

> Reemplaza los `TU_URL_RAW_*` por tus enlaces RAW reales. **Nota:** Actualizado! Podés usarlo directamente.

```sql
CREATE OR REPLACE VIEW customers AS
SELECT * FROM 'https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_customers.parquet';

CREATE OR REPLACE VIEW products AS
SELECT * FROM 'https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_order_items.parquet';

CREATE OR REPLACE VIEW orders AS
SELECT * FROM 'https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_orders.parquet';

CREATE OR REPLACE VIEW order_items AS
SELECT * FROM 'https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_payments.parquet';

CREATE OR REPLACE VIEW payments AS
SELECT * FROM 'https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_products.parquet';
```

Verifica que todo quedó bien:

```sql
SELECT 'customers' AS tabla, COUNT(*) AS filas FROM customers
UNION ALL
SELECT 'products', COUNT(*) FROM products
UNION ALL
SELECT 'orders', COUNT(*) FROM orders
UNION ALL
SELECT 'order_items', COUNT(*) FROM order_items
UNION ALL
SELECT 'payments', COUNT(*) FROM payments;
```


> **Mini-demo opcional (para enseñar el concepto con un dataset público):**
```sql
SELECT *
FROM read_csv_auto('https://raw.githubusercontent.com/tobilg/public-cloud-provider-ip-ranges/main/data/providers/all.csv')
LIMIT 10;
```

Esto prueba que DuckDB puede leer un CSV desde internet sin instalaciones.


## 1) Entendiendo el esquema relacional (20 min)

Hoy trabajaremos con un “mini-ecommerce” de libros:

- **customers**: clientes  
- **orders**: pedidos (encabezado)  
- **order_items**: detalle de productos por pedido  
- **products**: catálogo  
- **payments**: resumen de pago por pedido

### ¿Cómo se conectan?
- `customers.customer_id` → `orders.customer_id`  
- `orders.order_id` → `order_items.order_id`  
- `products.product_id` → `order_items.product_id`  
- `orders.order_id` → `payments.order_id`

> Idea clave: **una orden tiene muchas líneas (items)**. Por eso existe `order_items`.

### Ver “una muestra” de cada tabla
```sql
SELECT * FROM customers LIMIT 5;
SELECT * FROM products LIMIT 5;
SELECT * FROM orders LIMIT 5;
SELECT * FROM order_items LIMIT 5;
SELECT * FROM payments LIMIT 5;
```

### Ejercicio 1 (esquema)
1) Identifica, mirando las 5 tablas:  
- ¿Cuál columna parece el “id” principal en cada tabla?
- ¿Qué columnas parecen “conectar” tablas?

2) Escribe un comentario (en tu cuaderno) con estas relaciones:
- customers → orders
- orders → order_items
- products → order_items
- orders → payments


## 2) Llaves primarias, foráneas y relaciones (10 min)

### Definiciones rápidas
- **Llave primaria (PK):** identifica un registro de forma única.
- **Llave foránea (FK):** columna que “apunta” a la PK de otra tabla.

### ¿Por qué importa?
Porque en analítica, las relaciones te permiten responder preguntas como:
- “¿Cuánto gastan los clientes VIP?”
- “¿Qué categoría vende más?”
- “¿Qué canal trae más órdenes completadas?”

### Validaciones simples (sanity checks)
**(a) ¿Hay IDs duplicados en orders?**
```sql
SELECT order_id, COUNT(*) AS veces
FROM orders
GROUP BY order_id
HAVING COUNT(*) > 1;
```

**(b) ¿Hay items con product_id que no existe en products?**
```sql
SELECT oi.product_id, COUNT(*) AS items
FROM order_items oi
LEFT JOIN products p ON p.product_id = oi.product_id
WHERE p.product_id IS NULL
GROUP BY oi.product_id;
```

> Si estas consultas devuelven 0 filas, es una buena señal.

### Ejercicio 2
- Verifica si hay `customer_id` en `orders` que no exista en `customers`.
Pista: usa `LEFT JOIN` y filtra `WHERE c.customer_id IS NULL`.


## 3) Inspeccionar tablas con SQL (10 min)

En un mundo ideal siempre tienes un diccionario de datos.
En la vida real, muchas veces **lo construyes** inspeccionando.

### (a) Ver columnas rápidamente
En DuckDB puedes usar:

```sql
DESCRIBE customers;
DESCRIBE products;
DESCRIBE orders;
DESCRIBE order_items;
DESCRIBE payments;
```

### (b) Estadísticas rápidas
```sql
SELECT
  MIN(order_date) AS min_fecha,
  MAX(order_date) AS max_fecha,
  COUNT(*) AS total_ordenes
FROM orders;
```

### (c) Valores distintos (categorías, estados)
```sql
SELECT DISTINCT status FROM orders ORDER BY status;
SELECT DISTINCT category FROM products ORDER BY category;
SELECT DISTINCT segment FROM customers ORDER BY segment;
```


## 4) Consultas básicas: `SELECT`, `WHERE`, `ORDER BY` (20 min)

### 4.1 `SELECT` y `LIMIT`
```sql
SELECT * 
FROM customers
LIMIT 10;
```

### 4.2 Seleccionar columnas específicas
```sql
SELECT customer_id, customer_name, city, segment
FROM customers
LIMIT 10;
```

### 4.3 Filtrar con `WHERE`
Clientes de Bogotá:
```sql
SELECT customer_id, customer_name, city
FROM customers
WHERE city = 'Bogotá'
LIMIT 10;
```

Órdenes completadas:
```sql
SELECT order_id, customer_id, order_date, status
FROM orders
WHERE status = 'completed'
LIMIT 10;
```

### 4.4 Ordenar con `ORDER BY`
Top productos más caros:
```sql
SELECT product_id, title, category, price
FROM products
ORDER BY price DESC
LIMIT 10;
```

### 4.5 Filtros combinados (`AND`, `OR`)
```sql
SELECT order_id, order_date, status, payment_method
FROM orders
WHERE status = 'completed'
  AND payment_method = 'card'
ORDER BY order_date DESC
LIMIT 10;
```

### Ejercicio 3 (básico → medio)
1) Lista los 10 clientes **más recientes** (por `signup_date`).  
2) Lista las órdenes **no completadas** (todas excepto `completed`).  
   Pista: `WHERE status <> 'completed'`  
3) Muestra los productos de categoría `Tecnología` con precio > 80, ordenados del más caro al más barato.


## 5) Agrupar y agregar datos con `GROUP BY` (25 min)

### 5.1 Conteos por categoría
¿Cuántos productos hay por categoría?
```sql
SELECT
  category,
  COUNT(*) AS productos
FROM products
GROUP BY category
ORDER BY productos DESC;
```

### 5.2 KPI simple: órdenes por estado
```sql
SELECT
  status,
  COUNT(*) AS ordenes
FROM orders
GROUP BY status
ORDER BY ordenes DESC;
```

### 5.3 KPI de ingresos (desde payments)
Ingreso total pagado (ojo: incluye 0 en canceladas):
```sql
SELECT
  SUM(amount_paid) AS ingreso_total
FROM payments;
```

Ingreso por método de pago:
```sql
SELECT
  payment_method,
  SUM(amount_paid) AS ingreso
FROM payments
GROUP BY payment_method
ORDER BY ingreso DESC;
```

### 5.4 `HAVING` (filtrar grupos)
Categorías con al menos 10 productos:
```sql
SELECT
  category,
  COUNT(*) AS productos
FROM products
GROUP BY category
HAVING COUNT(*) >= 10
ORDER BY productos DESC;
```

### Ejercicio 4 (KPIs)
1) ¿Cuántas órdenes hay por **mes**?  
   Pista: usa `strftime(order_date, '%Y-%m')` o `date_trunc('month', order_date)`.  
2) ¿Cuál es el ingreso total por **estado** de orden?  
   Pista: une `orders` con `payments` por `order_id`.  
3) Top 5 ciudades por número de clientes.


## 6) Precisión con tipos y funciones (10–15 min)

### 6.1 Fechas
Si `order_date` viene como texto, puedes convertir:
```sql
SELECT
  CAST(order_date AS DATE) AS order_date_date
FROM orders
LIMIT 5;
```

Agrupar por mes con `date_trunc`:
```sql
SELECT
  date_trunc('month', CAST(order_date AS DATE)) AS mes,
  COUNT(*) AS ordenes
FROM orders
GROUP BY mes
ORDER BY mes;
```

### 6.2 Redondeo y decimales
Ticket promedio (AOV) sobre pagos:
```sql
SELECT
  ROUND(AVG(amount_paid), 2) AS aov
FROM payments;
```

### 6.3 Control de nulos
¿Cuántas órdenes tienen promo?
```sql
SELECT
  CASE WHEN promo_code IS NULL THEN 'sin_promo' ELSE 'con_promo' END AS promo_flag,
  COUNT(*) AS ordenes
FROM orders
GROUP BY promo_flag;
```


## 7) Buenas prácticas para SQL limpio (10 min)

### 7.1 Alias y formato
Evita esto:
```sql
SELECT * FROM orders, customers WHERE orders.customer_id = customers.customer_id;
```

Prefiere esto:
```sql
SELECT
  o.order_id,
  o.order_date,
  c.customer_name,
  c.city,
  o.status
FROM orders o
JOIN customers c ON c.customer_id = o.customer_id
WHERE o.status = 'completed'
ORDER BY o.order_date DESC
LIMIT 10;
```

### 7.2 Nombres claros
- Columnas: `snake_case`
- Alias de tablas: una letra o 2 letras (`c`, `o`, `oi`, `p`, `pay`)
- Evita `SELECT *` en consultas finales (úsalo para explorar)

### 7.3 Construye consultas por etapas
1) Explora
2) Filtra
3) Agrega
4) Ordena
5) Documenta (comentarios)


## Cierre
Hoy aprendiste a:

- Cargar CSV por URL con DuckDB (`read_csv_auto`)
- Explorar tablas, columnas y valores distintos
- Escribir filtros (`WHERE`) y ordenamientos (`ORDER BY`)
- Calcular métricas con agregaciones (`GROUP BY`, `HAVING`)
- Unir tablas con `JOIN` para preguntas analíticas reales

## Siguientes pasos
- Practica 15–20 min diarios escribiendo queries pequeñas.
- Intenta responder 3 preguntas:
  1) “¿Qué segmento genera más ingreso?”
  2) “¿Qué categorías tienen mejor margen?” (necesitarías `cost` desde products + items)
  3) “¿Qué canal trae más clientes VIP?”
